# Advent of Code - Day 9: Encoding Error

## Part One


With your neighbor happily enjoying their video game, you turn your attention to an open data port on the little screen in the seat in front of you.

Though the port is non-standard, you manage to connect it to your computer through the clever use of several paperclips. Upon connection, the port outputs a series of numbers (your puzzle input).

The data appears to be encrypted with the eXchange-Masking Addition System (XMAS) which, conveniently for you, is an old cypher with an important weakness.

XMAS starts by transmitting a preamble of 25 numbers. After that, each number you receive should be the sum of any two of the 25 immediately previous numbers. The two numbers will have different values, and there might be more than one such pair.

For example, suppose your preamble consists of the numbers 1 through 25 in a random order. To be valid, the next number must be the sum of two of those numbers:

- 26 would be a valid next number, as it could be 1 plus 25 (or many other pairs, like 2 and 24).
- 49 would be a valid next number, as it is the sum of 24 and 25.
- 100 would not be valid; no two of the previous 25 numbers sum to 100.
- 50 would also not be valid; although 25 appears in the previous 25 numbers, the two numbers in the pair must be different.

Suppose the 26th number is 45, and the first number (no longer an option, as it is more than 25 numbers ago) was 20. Now, for the next number to be valid, there needs to be some pair of numbers among 1-19, 21-25, or 45 that add up to it:

- 26 would still be a valid next number, as 1 and 25 are still within the previous 25 numbers.
- 65 would not be valid, as no two of the available numbers sum to it.
- 64 and 66 would both be valid, as they are the result of 19+45 and 21+45 respectively.

Here is a larger example which only considers the previous 5 numbers (and has a preamble of length 5):

```
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
```

In this example, after the 5-number preamble, almost every number is the sum of two of the previous 5 numbers; the only number that does not follow this rule is 127.

The first step of attacking the weakness in the XMAS data is to find the first number in the list (after the preamble) which is not the sum of two of the 25 numbers before it. **What is the first number that does not have this property?**

## Load and Clean Input data

:::{note}
The input data can be found [here](https://adventofcode.com/2020/day/9).
:::

In [ ]:
with open('../../data/advent-of-code/2020/day-9-input') as fid:
    data = fid.readlines()
    data = [int(x) for x in data]

print(len(data))
print(data[0:2])

In [ ]:
test_data = [
    35,
    20,
    15,
    25,
    47,
    40,
    62,
    55,
    65,
    95,
    102,
    117,
    150,
    182,
    127,
    219,
    299,
    277,
    309,
    576,
]

### Solution

In [3]:
def pairs_addup(data, target):
    n1, n2 = None, None
    for idx, num in enumerate(data):
        remain = target - num
        if remain in data[:idx] + data[idx + 1 :]:
            n1, n2 = num, remain
            break
    return n1, n2


def find_invalid_number(data, preamble_length, return_index=False):
    arr = data[preamble_length:]
    for idx, num in enumerate(arr):
        start, stop = idx, idx + preamble_length
        arr_to_check = data[start:stop]
        n1, n2 = pairs_addup(arr_to_check, num)
        if n1 is None:
            if return_index:
                return num, stop
            else:
                return num
    return None

In [4]:
print(f'{find_invalid_number(test_data, 5)} is the number that does not have the required property')

127 is the number that does not have the required property


In [5]:
print(f'{find_invalid_number(data, 25)} is the number that does not have the required property')

25918798 is the number that does not have the required property


## Part Two

The final step in breaking the XMAS encryption relies on the invalid number you just found: you must find a contiguous set of at least two numbers in your list which sum to the invalid number from step 1.

Again consider the above example:

```
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
```
In this list, adding up all of the numbers from 15 through 40 produces the invalid number from step 1, 127. (Of course, the contiguous set of numbers in your actual list might be much longer.)

To find the encryption weakness, add together the smallest and largest number in this contiguous range; in this example, these are 15 and 47, producing 62.

**What is the encryption weakness in your XMAS-encrypted list of numbers?**

### Solution

In [6]:
num = find_invalid_number(data, 25)
num

25918798

In [7]:
def find_contiguous_block(data, target):
    count = 0
    length = len(data)
    # Check each size from second index all the way up to the end of the array
    for idx in range(2, length):
        # At each size, shift the window from front to back
        for i in range(0, length - idx + 1):
            total = 0
            candidates = []
            # Add numbers in the computed window and check if they sum to the target
            for j in range(0, idx):
                total += data[i + j]
                candidates.append(data[i + j])

            # Does the total equal to the target?
            if total == target:
                min_num, max_num = min(candidates), max(candidates)
                return min_num, max_num

In [8]:
find_contiguous_block(test_data, 127)

(15, 47)

In [9]:
min_num, max_num = find_contiguous_block(data, num)
min_num, max_num

(1127699, 2213243)

In [10]:
print(f'The the encryption weakness in our XMAS-encrypted list of numbers is {min_num + max_num}')

The the encryption weakness in our XMAS-encrypted list of numbers is 3340942
